<h1>Applied Data Science Capstone Project Notebook</h1>

<h3>This notebook will be mainly used for the capstone project of "Applied Data Science Capstone" course of " Data Science Professional Certitificate" Program</h3>

In [1]:
!pip install folium
import folium
import pandas as pd
import numpy as np

import json
import requests

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [3]:
greek_cities_df=pd.read_html('http://www.tageo.com/index-e-gr-cities-GR.htm')[16]
greek_cities_df.rename(columns=greek_cities_df.iloc[0], inplace = True)
greek_cities_df.drop([0], inplace = True)
greek_cities_df.drop(columns=['Rank'], inplace = True)
greek_cities_df.rename(columns={'Latitude (DD)':'Latitude','Longitude (DD)':'Longitude','Population (2000)':'Population'}, inplace=True)
greek_cities_df['Population']=greek_cities_df['Population'].astype(str).astype(int)
greek_cities_df.head(50)

,City,Population,Latitude,Longitude
1,Athinai,762100,37.980,23.730
2,Thessaloniki,372100,40.640,22.940
3,Piraieus,179600,37.960,23.640
4,Patrai,164000,38.240,21.730
5,Peristerion,141000,38.020,23.700
6,Iraklion,133800,35.330,25.130
7,Larisa,127200,39.640,22.420
8,Kallithea,112100,37.950,23.700
9,Nikaia,95200,37.970,23.650
10,Kalamaria,89200,40.580,22.950


In [4]:
!pip install -U pandasql
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())


Requirement already up-to-date: pandasql in /opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages (0.7.3)


In [5]:


#aa=pysqldf("SELECT City,population, row_number() over (order by Population) as rn FROM greek_cities_df WHERE Population<'100000';")
#aa.head(50)

#bb=pysqldf("select  sum(population) from aa t1 ;")
#bb.head(50)

In [6]:
# create map
map_clusters =folium.Map(location=[37.98 ,23.73], zoom_start=11)


# add markers to the map
markers_colors = []
for lat, lon, poi in zip(greek_cities_df['Latitude'], greek_cities_df['Longitude'], greek_cities_df['City']):
    label = folium.Popup(str(poi), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
#map_clusters

### Get venues info from Foursquare

#### Define Foursqare Credentials

In [7]:
CLIENT_ID = 'ZWLQVDCUED4OJT423RUMHPFZGZWFOUDMA4M1DDFECN1OAA1C' 
CLIENT_SECRET = 'S4BS2TJ3YSXEAK22PI0GGEGA5OP3CX5VHUPRY0BOTBZ21HK5' 
VERSION = '20180605' 
LIMIT = 500

In [8]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
cities_venues_df = getNearbyVenues(names=greek_cities_df['City'],
                                   latitudes=greek_cities_df['Latitude'],
                                   longitudes=greek_cities_df['Longitude']
                                  )

Athinai
Thessaloniki
Piraieus
Patrai
Peristerion
Iraklion
Larisa
Kallithea
Nikaia
Kalamaria
Volos
Ilion
Glifada
Keratsinion
Zografos
Ilioupoli
Akharnai
Aigaleo
Nea smirni
Khalandrion
Amarousion
Koridallos
Nea ionia
Ayios dimitrios
Palaion faliron
Ioannina
Viron
Kavala
Galatsion
Ayia paraskevi
Serrai
Khalkis
Khania
Euosmon
Rodos
Katerini
Kalamata
Alexandroupoli
Trikala
Petroupoli
Lamia
Iraklion
Khaidarion
Xanthi
Kifisia
Komotini
Veroia
Drama
Agrinion
Sikeai
Stauroupolis
Ambelokipoi
Kalamakion
Polikhni
Kozani
Aryiroupoli
Ayioi anaryiroi
Kholargos
Karditsa
Nea ionia


In [10]:
cities_venues_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Athinai,37.980,23.730,Anäna,37.979214,23.729158,Café
1,Athinai,37.980,23.730,2 γουλιές & 2 μπουκιές,37.979393,23.730098,Coffee Shop
2,Athinai,37.980,23.730,Montagu (Μόντακιου),37.979523,23.732249,Coffee Shop
3,Athinai,37.980,23.730,The Clumsies,37.978988,23.729860,Cocktail Bar
4,Athinai,37.980,23.730,Paraphernalia,37.978476,23.730678,Furniture / Home Store


In [33]:
#cities_venues_df[cities_venues_df.Venue=='16 Bar']

bb=pysqldf("select  count(*), venue,'Venue Latitude' from cities_venues_df t1 group by venue,'Venue Latitude' having count(*)>1;")
bb.head(50)

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 7 - probably unsupported type.
[SQL: INSERT INTO cities_venues_df ("Neighbourhood", "Neighbourhood Latitude", "Neighbourhood Longitude", "Venue", "Venue Latitude", "Venue Longitude", "Venue Category", neib_cord, venue_cord, distance) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('Athinai', 37.98, 23.73, 'Anäna', 37.979214, 23.729158, 'Café', (37.98, 23.73), (37.979214, 23.729158), 0.11438941035679157), ('Athinai', 37.98, 23.73, '2 γουλιές & 2 μπουκιές', 37.97939306813599, 23.730097818463047, 'Coffee Shop', (37.98, 23.73), (37.97939306813599, 23.730097818463047), 0.06803023804483371), ('Athinai', 37.98, 23.73, 'Montagu (Μόντακιου)', 37.979522877688986, 23.73224855083927, 'Coffee Shop', (37.98, 23.73), (37.979522877688986, 23.73224855083927), 0.20409508171594665), ('Athinai', 37.98, 23.73, 'The Clumsies', 37.97898828627285, 23.729860381802997, 'Cocktail Bar', (37.98, 23.73), (37.97898828627285, 23.729860381802997), 

In [18]:
bb=pysqldf("select * from cities_venues_df t1 where venue in ('Barley');")
bb.head(50)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Larisa,39.640,22.420,Barley,39.641679,22.416614,Bar
1,Euosmon,40.670,22.910,Barley,40.667925,22.911064,Bar


In [23]:
!pip install haversine
from haversine import haversine,haversine_vector, Unit


In [22]:
import math

def haversine(coord1, coord2):
    R = 6372800  # Earth radius in meters
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    phi1, phi2 = math.radians(lat1), math.radians(lat2) 
    dphi       = math.radians(lat2 - lat1)
    dlambda    = math.radians(lon2 - lon1)
    
    a = math.sin(dphi/2)**2 + \
        math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    

    return 2*R*math.atan2(math.sqrt(a), math.sqrt(1 - a))

In [24]:
cities_venues_df['Neighbourhood Latitude']=cities_venues_df['Neighbourhood Latitude'].astype(str).astype(float)
cities_venues_df['Neighbourhood Longitude']=cities_venues_df['Neighbourhood Longitude'].astype(str).astype(float)

cities_venues_df['neib_cord'] =list(zip(cities_venues_df['Neighbourhood Latitude'], cities_venues_df['Neighbourhood Longitude']))
cities_venues_df['venue_cord']=list(zip(cities_venues_df['Venue Latitude'], cities_venues_df['Venue Longitude']))

#cities_venues_df['distance']=cities_venues_df.apply(haversine(cities_venues_df['neib_cord'],cities_venues_df['venue_cord']))

In [25]:
cities_venues_df['distance'] = cities_venues_df.apply(lambda row : haversine(row['neib_cord'],  row['venue_cord']), axis = 1) 

 
   # print('\nAfter Applying Function: ') 
    # printing the new dataframe 
cities_venues_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,neib_cord,venue_cord,distance
0,Athinai,37.98,23.73,Anäna,37.979214,23.729158,Café,"(37.98, 23.73)","(37.979214, 23.729158)",0.114389
1,Athinai,37.98,23.73,2 γουλιές & 2 μπουκιές,37.979393,23.730098,Coffee Shop,"(37.98, 23.73)","(37.97939306813599, 23.730097818463047)",0.068030
2,Athinai,37.98,23.73,Montagu (Μόντακιου),37.979523,23.732249,Coffee Shop,"(37.98, 23.73)","(37.979522877688986, 23.73224855083927)",0.204095
3,Athinai,37.98,23.73,The Clumsies,37.978988,23.729860,Cocktail Bar,"(37.98, 23.73)","(37.97898828627285, 23.729860381802997)",0.113161
4,Athinai,37.98,23.73,Paraphernalia,37.978476,23.730678,Furniture / Home Store,"(37.98, 23.73)","(37.978475712062284, 23.73067756282196)",0.179596


In [32]:
cities_venues_df[cities_venues_df.Venue=='Barley']



,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,neib_cord,venue_cord,distance
355,Larisa,39.64,22.42,Barley,39.641679,22.416614,Bar,"(39.64, 22.42)","(39.64167869091645, 22.416614078267703)",0.344818
1229,Euosmon,40.67,22.91,Barley,40.667925,22.911064,Bar,"(40.67, 22.91)","(40.66792480076344, 22.911064463903593)",0.247601


In [ ]:
#!pip install haversine
#haversine(37.980,23.730,37.979214,23.729158)
#cities_venues_df['distance']=cities_venues_df.apply(lambda x: haversine((x.cities_venues_df['Neighbourhood Latitude'], x.cities_venues_df['Neighbourhood Longitude']),(x.cities_venues_df['Venue Latitude'],x.cities_venues_df['Venue Longitude'])), axis=1)

from haversine import haversine_vector, Unit

cities_venues_df['Venue Latitude']=cities_venues_df['Venue Latitude'].astype(str).astype(float)
cities_venues_df['Venue Longitude']=cities_venues_df['Venue Longitude'].astype(str).astype(float)

coord1=(cities_venues_df['Neighbourhood Latitude'],cities_venues_df['Neighbourhood Longitude']).astype(float)
coord2=(cities_venues_df['Venue Latitude'], cities_venues_df['Venue Longitude']).astype(float)

coord1

lyon = (45.7597, 4.8422) # (lat, lon)
paris = (48.8567, 2.3508)
new_york = (40.7033962, -74.2351462)

haversine((cities_venues_df['Neighbourhood Latitude'].astype(float),cities_venues_df['Neighbourhood Longitude'].astype(float)),(cities_venues_df['Venue Latitude'].astype(float), cities_venues_df['Venue Longitude'].astype(float)))